# Prueba técnica - Data engineer

## Objetivo
Constriur una solución End-to-End que cubra:
- Ingesta
- Limpieza
- Modelado
- Exposición vía API
- Visualización

En este archivo se muestra el desarrollo a través de:
- Proceso de limpieza
- Validación de datos
- Consumo de la API
- Generación de gráficas

Esta Notebook muestra algunas de las etapas de análisis, para comprender la data y a partir de eso decidir la manera en que se abordaría en la solución final. Para correr esta Notebook sin problema, el contenedor debe estar corriendo.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests

Se lee el archivo .csv y se leen las primeras cinco filas.

In [2]:
df=pd.read_csv("../data/raw/2026012_ data.csv")
df.head()

,invoice_id,issue_date,customer_id,customer_name,item_description,qty,unit_price,total,status
0,INV-00001,2023-08-26,C-105,stark ind,Server Setup,14,1200.50,16807.0,Processing
1,INV-00002,2023-09-16,C-104,Umbrella Corp,Audit Service,2,2000.00,4000.0,REFUNDED
2,INV-00003,2023/04/23,C-102,Soylent Corp,Licencia Software,11,5000.00,55000.0,Cancelled
3,INV-00004,2023-03-22,C-108,Massive Dynamic,Front-end dev,3,50.00,150.0,Cancelled
4,INV-00005,2023/11/25,C-105,Stark Ind,Consultoria Data,14,1500.00,21000.0,Cancelled


Se leen las características de las columnas

In [3]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   invoice_id        9950 non-null   str    
 1   issue_date        10000 non-null  str    
 2   customer_id       10000 non-null  str    
 3   customer_name     9513 non-null   str    
 4   item_description  10000 non-null  str    
 5   qty               10000 non-null  int64  
 6   unit_price        10000 non-null  str    
 7   total             9811 non-null   float64
 8   status            10000 non-null  str    
dtypes: float64(1), int64(1), str(7)
memory usage: 1.3 MB


Junto con lo anterior, se ven los diferentes valores estadísticos, que ayudan a explorar el dataset.

In [4]:
df.describe(include="all")

,invoice_id,issue_date,customer_id,customer_name,item_description,qty,unit_price,total,status
count,9950,10000,10000,9513,10000,10000.000000,10000,9811.000000,10000
unique,9859,1098,11,33,8,NaN,24,NaN,5
top,INV-02852,2023-08-08,C-103,Initech,Server Setup,NaN,1200.50,NaN,Paid
freq,3,35,946,755,1332,NaN,1155,NaN,2016
mean,NaN,NaN,NaN,NaN,NaN,10.425900,NaN,13572.831974,NaN
std,NaN,NaN,NaN,NaN,NaN,5.768989,NaN,20084.028607,NaN
min,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,19.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,5.000000,NaN,700.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,10.000000,NaN,5500.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,15.000000,NaN,19208.000000,NaN


Junto con esto, se muestra la media de cada columna, lo que ayuda a decidir el tipo de dato más adelante.

In [5]:
df.isna().mean().sort_values(ascending=False)

customer_name       0.0487
total               0.0189
invoice_id          0.0050
customer_id         0.0000
issue_date          0.0000
item_description    0.0000
qty                 0.0000
unit_price          0.0000
status              0.0000
dtype: float64

Se revisan tickets duplicados

In [6]:
df.duplicated(subset=["invoice_id"]).sum()

np.int64(140)

## Limpieza y normalización

Función auxiliar que usa expresiones regulares para limpiar errores numéricos.*

In [7]:
def clean_numeric(series: pd.Series)->pd.Series:
    return (series.astype(str).str.replace(r"[^\d\.-]", "", regex=True).replace("", pd.NA).astype(float))

Se fuerza el tipo de dato fecha en la columna "issue_date"

In [8]:
df["issue_date"]=pd.to_datetime(df["issue_date"], errors="coerce").dt.date

Se limpian los datos numéricos para las columnas "qty", "unit_price", "total". El  nombre y la exploración anterior sugieren que estas columnas serían numéricas.

In [9]:
num_cols=["qty", "unit_price", "total"]
for col in num_cols:
    df[col]=clean_numeric(df[col])

In [10]:
df.head()

,invoice_id,issue_date,customer_id,customer_name,item_description,qty,unit_price,total,status
0,INV-00001,2023-08-26,C-105,stark ind,Server Setup,14.0,1200.5,16807.0,Processing
1,INV-00002,2023-09-16,C-104,Umbrella Corp,Audit Service,2.0,2000.0,4000.0,REFUNDED
2,INV-00003,NaT,C-102,Soylent Corp,Licencia Software,11.0,5000.0,55000.0,Cancelled
3,INV-00004,2023-03-22,C-108,Massive Dynamic,Front-end dev,3.0,50.0,150.0,Cancelled
4,INV-00005,NaT,C-105,Stark Ind,Consultoria Data,14.0,1500.0,21000.0,Cancelled


Se verifica el estado actual de esas columnas.

In [11]:
df[num_cols].isna().mean()

qty           0.0000
unit_price    0.0000
total         0.0189
dtype: float64

Se recalculan los totales.

In [12]:
df["recalculated_total"]=df["qty"]*df["unit_price"]
(df["recalculated_total"]-df["total"]).abs().describe()

count    9811.000000
mean        0.019366
std         0.137815
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
dtype: float64

In [15]:
df["total_calc"]=df["qty"]*df["unit_price"]
df["total"]=df["total"].fillna(df["total_calc"])

In [13]:
BASE_URL="htpp://localhost:8000"

In [16]:
response=requests.get(f"{BASE_URL}/sales/monthly")
sales_monthly=pd.DataFrame(response.json())
sales_monthly.head()

InvalidSchema: No connection adapters were found for 'htpp://localhost:8000/sales/monthly'

In [17]:
sales_monthly["date"] = pd.to_datetime(
    sales_monthly["year"].astype(str) + "-" +
    sales_monthly["month"].astype(str).str.zfill(2) + "-01"
)

sales_monthly = sales_monthly.sort_values("date")

plt.figure()
plt.plot(sales_monthly["date"], sales_monthly["total_sales"])
plt.title("Tendencia histórica de ventas (API)")
plt.xticks(rotation=45)
plt.grid(True)
plt.show()


NameError: name 'sales_monthly' is not defined

In [ ]:
response = requests.get(f"{BASE_URL}/sales/top")
top_customers = pd.DataFrame(response.json())

top_customers


In [ ]:
plt.figure()
plt.bar(top_customers["customer_name"], top_customers["total_sales"])
plt.title("Top 5 clientes por facturación (API)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
